In [51]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

np.set_printoptions(6, suppress=True)

plt.style.use([
    'grid',
    'notebook'
])

In [52]:
A_vertices = []
B_vertices = []

Ts = 4

r = .31
mu = .4
sigma = .55
A2 = np.pi * r**2

Kb = 15.92e-4

In [53]:
h1_interval = np.linspace(20, 74, 1000)
h2_interval = np.linspace(12, 50, 1000)

Z1 = []
Z2 = []
Z3 = []

min_diff = 6

for i in range(len(h1_interval)):
    h1 = h1_interval[i]
    
    for j in range(i, len(h1_interval)): 
        h2 = h2_interval[j]

        diff = h1 - h2

        if diff < min_diff:
            continue

        R12 = (0.2371*diff+6.9192)*10
        q0  = (18.6367*h2+700.6831)*1e-4
        a1  = (3*r/5)*(2.7*r-((np.cos(2.5*np.pi*(h1-8)*1e-2-mu))/(sigma*np.sqrt(2*np.pi)))*np.exp(-(((h1-8)*1e-2-mu)**2)/(2*sigma**2)))

        z1 = Ts/R12
        z2 = (Ts*q0)/h2
        z3 = 1/a1

        Z1.append(z1)
        Z2.append(z2)
        Z3.append(z3)


min_z1, max_z1 = min(Z1), max(Z1)
min_z2, max_z2 = min(Z2), max(Z2)
min_z3, max_z3 = min(Z3), max(Z3)

print(min_z1, max_z1)
print(min_z2, max_z2)
print(min_z3, max_z3)

0.0317218627077782 0.04795128149799804
0.0130601448 0.03081078333333333
3.4476944509493617 18.456983136119643


In [54]:
z1_bounds = np.array([min_z1, max_z1])
z2_bounds = np.array([min_z2, max_z2])
z3_bounds = np.array([min_z3, max_z3])

np.save('./data/z1_bounds.npy', z1_bounds)
np.save('./data/z2_bounds.npy', z2_bounds)
np.save('./data/z3_bounds.npy', z3_bounds)

for z1 in z1_bounds:
    for z2 in z2_bounds:
        for z3 in z3_bounds:
            Ai = np.array([
                [1 - z1*z3, z1*z3],
                [z1/A2, 1 - (z1+z2)/A2]
            ])

            Bi = np.array([
                [Ts*Kb*z3],
                [.0]
            ])

            print(
                np.abs(np.linalg.eigvals(Ai))
            )

            A_vertices.append(Ai)
            B_vertices.append(Bi)

[0.980105 0.762197]
[0.302491 0.963688]
[0.959569 0.723938]
[0.29175  0.915635]
[0.979377 0.653214]
[0.050684 0.963561]
[0.955838 0.617959]
[0.060771 0.914853]


In [55]:
Z1_, Z2_, Z3_ = sp.symbols('Z1 Z2 Z3')

z1_min, z1_max = z1_bounds
z2_min, z2_max = z2_bounds
z3_min, z3_max = z3_bounds

M1 = (z1_max - Z1_) / (z1_max - z1_min)
N1 = (z2_max - Z2_) / (z2_max - z2_min)
P1 = (z3_max - Z3_) / (z3_max - z3_min)

M2 = 1 - M1
N2 = 1 - N1
P2 = 1 - P1

In [56]:
print(' ')
print(M1)
print(N1)
print(P1)
print(' ')
print(M2)
print(N2)
print(P2)

 
2.95459018698158 - 61.6165010544074*Z1
1.73575633774947 - 56.3360015540925*Z2
1.22970405348761 - 0.0666254091699919*Z3
 
61.6165010544074*Z1 - 1.95459018698158
56.3360015540925*Z2 - 0.735756337749472
0.0666254091699919*Z3 - 0.229704053487612


In [57]:
for i in range(len(A_vertices)):
    A = A_vertices[i]
    B = B_vertices[i]
    
    np.save(f'./data/vertices/A_{i}', A)
    np.save(f'./data/vertices/B_{i}', B)

In [58]:
from scipy.io import savemat

mats_dict = {}

for i in range(len(A_vertices)):
    A = A_vertices[i]
    B = B_vertices[i]
    
    mats_dict[f'A{i}'] = A
    mats_dict[f'B{i}'] = B

savemat(f'./data/vertices_{Ts}_Ts.mat', mats_dict)